In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/kubeflow/pipelines/blob/master/components/google-cloud/google_cloud_pipeline_components/experimental/tensorflow_probability/anomaly_detection/tfp_anomaly_detection.ipynb"">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/kubeflow/pipelines/blob/master/components/google-cloud/google_cloud_pipeline_components/experimental/tensorflow_probability/anomaly_detection/tfp_anomaly_detection.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

# Anomaly Detection with TensorFlow Probability STS on Kubeflow Pipelines

## Overview

This notebook demonstrates how to use [TensorFlow Probability](https://www.tensorflow.org/probability) and [Kubeflow Pipelines](https://www.kubeflow.org/docs/components/pipelines/) for anomaly detection in time series data. It uses structural time series (STS), a class of Bayesian statistical models, to decompose a time series into interpretable seasonal and trend components. This algorithm fits an STS model to the time series, generates a forecast of acceptable values for each timestep, and flags any points outside of the forecast as an anomaly. To learn more about STS models, check out this demo on [Structural Time Series Modeling Case Studies](https://www.tensorflow.org/probability/examples/Structural_Time_Series_Modeling_Case_Studies_Atmospheric_CO2_and_Electricity_Demand).

This demo is most relevant for those who would like to automatically flag anomalies in time series data and can be used for applications like network monitoring, infrastructure maintenance, and sales tracking.

### Dataset

{TODO: Replace synthetic data with benchmark}

This demo uses the [Numenta Anomaly Benchmark](https://github.com/numenta/NAB), a popular benchmark of time series data with labelled anomalies. More specifically, our demo uses [nyc_taxi.csv](https://github.com/numenta/NAB/blob/d2854d17a3feb9e143b1e9a715c5af67da2c1888/data/realKnownCause/nyc_taxi.csv) which reports the number of NYC taxi passengers over time.

### Objective

In this notebook, you will learn how to:

* Infer the frequency of a time series and regularize it with `tfp.sts.regularize_series`
* Infer the the structure of the time series using `tensorflow_probability.python.sts.default_model.build_default_model`
* Fit the model with variational inference using `tfp.vi.fit_surrogate_posterior`
* Identify anomalies based on the predictive distribution of acceptable values at each timestep
* Launch the anomaly detection algorithm on Kubeflow Pipelines

### Costs
This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Google Cloud Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip3 install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Install additional packages

Install additional package dependencies not installed in your notebook environment.

In [ ]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [ ]:
! pip3 install {USER_FLAG} --upgrade kfp
! pip3 install {USER_FLAG} --upgrade google-cloud-pipeline-components
! pip3 install {USER_FLAG} --upgrade tensorflow
! pip3 install {USER_FLAG} --upgrade matplotlib
! pip3 install {USER_FLAG} --upgrade numpy
! pip3 install {USER_FLAG} --upgrade pandas

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API, Cloud Build API, Cloud Storage API, and Container Registry API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,cloudbuild.googleapis.com,storage.googleapis.com,containerregistry.googleapis.com).

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
import os

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
  shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
  PROJECT_ID = shell_output[0]
  print("Project ID: ", PROJECT_ID)

Project ID:  


Otherwise, set your project ID here.

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
  PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you submit a training job, Vertex AI saves all resources to the given GCS bucket.

We will also use the same bucket to download and host the input data 

Set the name of your Cloud Storage bucket below. It must be unique across all
Cloud Storage buckets.

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where Vertex AI services are
available](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions). You may not use a Multi-Regional Storage bucket for training with Vertex AI.

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}
REGION = "[your-region]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_NAME

### Import libraries and define constants

In [ ]:
PIPELINE_NAME = '{0}-{1}'.format('tfp-anomaly-detection', TIMESTAMP)
PIPELINE_ROOT = '{0}/{1}'.format(BUCKET_NAME, PIPELINE_NAME)

In [ ]:
from typing import Callable, Optional, Mapping, Any

import kfp
from kfp.v2 import compiler
from kfp.v2 import dsl
from kfp.v2.google.client import AIPlatformClient
from kfp.v2.dsl import Dataset

### Define the anomaly detection component

In [ ]:
train_op = kfp.components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/master/components/google-cloud/google_cloud_pipeline_components/experimental/tensorflow_probability/anomaly_detection/component.yaml')

### Define the pipeline

In [ ]:
@dsl.pipeline(
    pipeline_root=PIPELINE_ROOT, name=PIPELINE_NAME)
def pipeline(input_url: str) -> None:
  """
      Train model and return detected anomalies.
  """
  input_task = kfp.dsl.importer(
        artifact_uri=input_url,
        artifact_class=Dataset)
  train_task = train_op(input_dataset=input_task.output).set_memory_limit('50G')

In [ ]:
def run_pipeline(pipeline: Callable,
                 parameter_values: Optional[Mapping[str, Any]] = {},
                 enable_caching: bool = False) -> None:
  """Runs a given pipeline function using Kubeflow Pipelines.

  Args:
   pipeline: The function to run.
   parameter_values: Parameters passed to the pipeline function when run.
   enable_caching: Whether to used cached results from previous runs.
  """
  compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path='{}_pipeline.json'.format(PIPELINE_NAME))

  api_client = AIPlatformClient(
    project_id=PROJECT_ID,
    region=REGION,
  )

  _ = api_client.create_run_from_job_spec(
    job_spec_path='{}_pipeline.json'.format(PIPELINE_NAME),
    pipeline_root=PIPELINE_ROOT,
    parameter_values=parameter_values,
    enable_caching=enable_caching)

### Download the data

In [ ]:
import os

# TODO: Do an efficient check for if files exist in GCS
NAB_DATA_BLOB = '{0}/NAB'.format(BUCKET_NAME)
if not os.path.exists('content/NAB'):
  !git clone https://github.com/numenta/NAB
!gsutil cp -r NAB/data $NAB_DATA_BLOB

In [ ]:
import tensorflow as tf

chosen_task = 'nyc_taxi.csv'
nab_files = tf.io.gfile.glob('{0}/*/*.csv'.format(NAB_DATA_BLOB))
chosen_task_file = [file for file in nab_files if chosen_task in file][0]
print('Running the pipeline on {0}'.format(chosen_task))

In [ ]:
parameter_values = {
  'input_url': chosen_task_file
}
run_pipeline(pipeline, parameter_values=parameter_values)

### Visualize the results

In [ ]:
# Copy output from pipeline to download locally in Colab
output_file = '[your-pipeline-output]' # @param {type:'string'}
!gsutil cp $output_file .
!wget https://raw.githubusercontent.com/numenta/NAB/master/labels/combined_labels.json

In [ ]:
from matplotlib import pylab as plt
from matplotlib.lines import Line2D
import pandas as pd
import numpy as np
import json

In [ ]:
def plot_predictions(predictions: pd.DataFrame) -> None:
  """
    Plots the time series, forecast, detected anomalies, and residuals.

    Args:
      predictions: The output of the anomaly detection algorithm.
  """
  timestamp = pd.to_datetime(predictions['timestamp'], format='%Y-%m-%d')
  value = np.array(predictions['value'])
  lower_limit = np.array(predictions['lower_limit'])
  upper_limit = np.array(predictions['upper_limit'])
  mean = np.array(predictions['mean'])
  anomalies = np.array(predictions['label']).nonzero()[0]
  targets = []
  if 'target' in predictions:
    targets = np.array(predictions['target']).nonzero()[0]

  fig = plt.figure(figsize=(10, 5), constrained_layout=True)
  spec = fig.add_gridspec(ncols=1, nrows=2, height_ratios=[2., 1.])
  series_ax = fig.add_subplot(spec[0, 0])
  residuals_ax = fig.add_subplot(spec[1, 0], sharex=series_ax)

  # Plot anomalies on series_ax
  series_ax.plot(
      timestamp,
      value,
      color='black',
      alpha=0.6)
  series_ax.fill_between(
      timestamp,
      lower_limit,
      upper_limit,
      color='tab:blue',
      alpha=0.3)

  for anomaly_idx in anomalies:
    x = timestamp[anomaly_idx]
    y = value[anomaly_idx]
    series_ax.scatter(x, y, s=100, alpha=0.4, c='red')
  
  for target_idx in targets:
    x = timestamp[target_idx]
    y = value[target_idx]
    series_ax.scatter(x, y, s=100, alpha=0.4, c='green')
    series_ax.annotate(x, (x, y))

  # Plot residuals on residuals_ax
  time_delta = timestamp[1] - timestamp[0]
  residuals_ax.bar(
      timestamp,
      height=upper_limit - lower_limit,
      bottom=lower_limit - mean,
      width=time_delta,
      align='center',
      color='tab:blue',
      alpha=0.3)
  residuals_ax.bar(
      timestamp,
      width=time_delta,
      height=value - mean,
      align='center',
      color='black',
      alpha=0.6)

  # Set up grid styling
  series_ax.set_ylabel('Original series')
  residuals_ax.set_ylabel('Residuals')
  series_ax.grid(True, color='whitesmoke')
  residuals_ax.grid(True, color='whitesmoke')
  series_ax.set_axisbelow(True)
  residuals_ax.set_axisbelow(True)

  # Add title and legend
  series_ax.set_title('TFP STS model forecast, anomalies, and residuals for {0}'.format(chosen_task))
  create_legend_label = lambda label, color: Line2D([0], [0], marker='o', color='w', label=label, markerfacecolor=color, markersize=10)
  legend_elements = [create_legend_label(label, color) for label, color in [('predicted anomaly', 'red'), ('target', 'green')]]
  series_ax.legend(handles=legend_elements, loc='lower right')

In [ ]:
predictions = pd.read_csv('/content/output_dataset')
targets = json.load(open('/content/combined_labels.json'))

In [ ]:
# Add targets to the predictions for plotting
chosen_task_targets = set([])
for key in targets:
  if chosen_task in key:
    chosen_task_targets = set(targets[key])
    
predictions['target'] = predictions.apply(lambda df: df['timestamp'] in chosen_task_targets, axis=1)

In [ ]:
plot_predictions(predictions)